In [1]:
import numpy as np
import pandas as pd 
import networkx as nx
import math
import sympy as sp
import matplotlib.pyplot as plt
import gc
import time

from scipy import linalg as LA
from sympy import *
from numpy import linalg as LA
from numpy.linalg import matrix_rank

In [2]:
#Establecer directorio de datos

DIR='/Users/I_am_AlexX/Downloads/Telegram Desktop'
FILE='/adjmatrix_UNS.txt'

file='{}{}'.format(DIR,FILE)

print('File directory:{}'.format(file))

File directory:/Users/I_am_AlexX/Downloads/Telegram Desktop/adjmatrix_UNS.txt


In [3]:
data=pd.read_table(file)

In [4]:
a=np.diag(np.ones(16748),0)
A=data-a

In [5]:
del data

In [6]:
del a

In [8]:
#Se obtiene el mismo resultado 

threshold= A.mean().mean()
threshold

0.08567775225309081

In [9]:
#Función para ver qué forma tiene la matriz del data set que estamos
#estudiando 

def mat_shape(mat,n):
    
    new_data=mat.iloc[:n,:n]
    
    new_data2=new_data.apply(lambda x:(np.where(x<threshold,0,x)))
    adj_mat=new_data2.apply(lambda x:(np.where(x>threshold,1,x)))
    
    #completar la matriz triangular superior para tener la forma completa de una matriz
    # simétrica
    
    adj_mod=np.matrix(adj_mat.fillna(0))
    adj_mod_trans=adj_mod.transpose()
    simm_mat=adj_mod+adj_mod_trans
    
    
    return simm_mat

In [10]:

#Función para contruir la matriz Laplaciana a partir de la matriz de adyacencia
def lap_mat(mat,n):
   
    simm_mat = mat_shape(mat,n)
    
    Adj_mod=nx.from_numpy_matrix(simm_mat)
    L=nx.laplacian_matrix(Adj_mod)
    L_matrix=L.toarray()
    
    return L_matrix

In [11]:
#Función para obtener los valores y vectores propios de la matriz laplaciana a partir de la matriz de adyacencia
# como parámetro


def eig_val_from_lap(mat,n):
    
    inicio = time.perf_counter()
    
    simm_mat = mat_shape(mat,n)
    
    Adj_to_nx=nx.from_numpy_matrix(simm_mat)
    L_from_Adj=nx.laplacian_matrix(Adj_to_nx)
    L_arr=L_from_Adj.toarray()
    
    L_np=np.matrix(L_arr)
    
    _vals, e_vecs = LA.eig(L_np)
    
    final = time.perf_counter()
    
    return _vals#, print("El tiempo de ejecución fue, ", final- inicio ,"segundos")#,(e_vecs) 

In [12]:
#Función para contruir la matriz de grado a partir de la de la matriz de adyacencia

def degree_mat_from_adj(mat,n):
    
    simm_mat = mat_shape(mat, n)
     
    Adj_to_nx=nx.from_numpy_matrix(simm_mat)
    
    L_from_Adj=nx.laplacian_matrix(Adj_to_nx) #obtener la Laplaciana asociada a la de adyacencia
    L_arr=L_from_Adj.toarray()
    L_np=np.matrix(L_arr)
    
    deg_Mat=L_np+simm_mat
    
    return deg_Mat

In [13]:
#La siguiente función sirve para obtener los elementos de la diagonal de la matriz de
# grado y la guarda en un array llamado out_p


def get_elem_in_diag(Mat):
    Mat_mod = np.squeeze(np.asarray(Mat))#convierte numpy matrix en array
    out_p=[]
    for elem in np.nditer(Mat_mod): #nditer itera sobre arrays
        if elem != 0: # omite los valores 0 en el array pues la matriz de grado fuera de la diagonal tiene solo ceros
            out_p.append(int(elem))
            
    return out_p

In [14]:
#Función para obtener el número mayor de una lista de números dentro de un array

def func(a,N):
    return np.argsort(a)[::-1][:N]

In [15]:
# El siguiente módulo, crea un array con los primeros "n" genes que deseen obtener del
# Data Frame "data"

#Parametros: data , n 

def get_genes(data,n):
    
    genes_arr = sorted(data) #obtener un array que contiene los elementos del header del DataFrame
    num_genes_extracted = range(n)
    genes = np.array(genes_arr)[num_genes_extracted]
    
    return genes

In [16]:
#Este script es capaz de dibujar el grafo con sus vértices respectivos etiquetados

#Este módulo ha sifo modificado para que sea capaz de etiquetar los nombres de los genes 
#involucrados

def plt_with_label(mat,n):
    
    simm_mat = mat_shape(mat, n)
    
    adj_to_graph=nx.from_numpy_matrix(simm_mat)
    labelmap=dict(zip(adj_to_graph.nodes(),get_genes(data,n)))
    
    plt.figure(1,figsize=(16,16))
    
    nx.draw(adj_to_graph, labels=labelmap, with_labels=True,
    font_size=15,font_weight='bold',font_color='black',edge_color='gray',node_color='green',node_size=200)
    
    graph_with_lab=plt.show()
    
    return graph_with_lab

In [17]:
# función para ordenar los elementos de un array de menor a mayor según su valor
def sort_arr(x):
    for i in range(len(x)): # se debe iterar sobre todos los elementos del array
        swap = i + np.argmin(x[i:]) # cada que se completa un ciclo del bucle vamos "eliminando" el primer elemento para el siguiente análisis
        (x[i], x[swap]) = (x[swap], x[i]) # se intercambian valores para que el número menor se ponga en el primer lugar del array para el siguiente análisis
    return x

# Análisis de conectividad

## 1000 genes

In [16]:
sort_arr(eig_val_from_lap(A,1000))

array([-3.41244068e-13,  1.68523007e+02,  1.74778470e+02,  1.79539583e+02,
        1.82837235e+02,  1.85071413e+02,  1.85721432e+02,  1.90628835e+02,
        1.92189059e+02,  1.94208244e+02,  1.95785831e+02,  1.97616632e+02,
        1.98059715e+02,  1.98613472e+02,  1.99729934e+02,  2.00444605e+02,
        2.01467145e+02,  2.02247269e+02,  2.02891684e+02,  2.04137318e+02,
        2.04927938e+02,  2.05774466e+02,  2.06737107e+02,  2.07361613e+02,
        2.08195352e+02,  2.08468779e+02,  2.09672859e+02,  2.10136086e+02,
        2.11025132e+02,  2.12688592e+02,  2.13079177e+02,  2.14752073e+02,
        2.15185629e+02,  2.15666994e+02,  2.16702520e+02,  2.16906468e+02,
        2.17432146e+02,  2.18617809e+02,  2.18881417e+02,  2.20203169e+02,
        2.23255537e+02,  2.23345766e+02,  2.24053402e+02,  2.24809509e+02,
        2.25582899e+02,  2.26623777e+02,  2.27213295e+02,  2.27633559e+02,
        2.28572970e+02,  2.29137215e+02,  2.30041525e+02,  2.30286903e+02,
        2.31108114e+02,  

In [17]:
sort_arr(get_elem_in_diag(degree_mat_from_adj(A,1000)))

[170,
 177,
 183,
 184,
 186,
 187,
 192,
 195,
 198,
 199,
 200,
 200,
 200,
 200,
 203,
 204,
 205,
 205,
 206,
 208,
 208,
 209,
 209,
 210,
 210,
 211,
 212,
 212,
 213,
 214,
 217,
 217,
 218,
 218,
 219,
 219,
 219,
 221,
 221,
 226,
 227,
 227,
 227,
 229,
 230,
 230,
 230,
 230,
 230,
 231,
 231,
 233,
 233,
 236,
 237,
 237,
 237,
 238,
 238,
 238,
 239,
 239,
 240,
 240,
 240,
 241,
 241,
 241,
 242,
 242,
 244,
 246,
 247,
 247,
 247,
 247,
 248,
 249,
 249,
 250,
 250,
 251,
 251,
 252,
 252,
 253,
 253,
 253,
 253,
 253,
 254,
 255,
 255,
 256,
 256,
 257,
 257,
 257,
 258,
 258,
 258,
 259,
 259,
 260,
 260,
 260,
 260,
 260,
 261,
 261,
 262,
 262,
 264,
 265,
 267,
 268,
 268,
 269,
 269,
 269,
 271,
 271,
 272,
 273,
 273,
 274,
 274,
 275,
 275,
 275,
 275,
 276,
 276,
 276,
 276,
 277,
 277,
 277,
 278,
 278,
 278,
 280,
 280,
 280,
 281,
 281,
 281,
 282,
 282,
 282,
 282,
 283,
 283,
 284,
 284,
 284,
 284,
 284,
 284,
 285,
 285,
 285,
 286,
 287,
 287,
 287,
 287

## 2000 genes

In [19]:
sort_arr(eig_val_from_lap(A,2000))

array([-6.87337753e-13,  2.91139092e+02,  3.04064971e+02, ...,
        1.27034609e+03,  1.27339331e+03,  1.29830296e+03])

In [20]:
sort_arr(get_elem_in_diag(degree_mat_from_adj(A,2000)))

[292,
 305,
 346,
 353,
 357,
 362,
 373,
 374,
 375,
 375,
 380,
 381,
 382,
 382,
 385,
 387,
 387,
 387,
 389,
 390,
 391,
 392,
 399,
 400,
 401,
 402,
 402,
 402,
 403,
 403,
 404,
 404,
 404,
 404,
 406,
 406,
 407,
 408,
 408,
 412,
 414,
 415,
 415,
 416,
 417,
 419,
 420,
 421,
 421,
 424,
 425,
 426,
 429,
 429,
 431,
 431,
 431,
 432,
 434,
 434,
 435,
 436,
 439,
 442,
 443,
 443,
 443,
 446,
 446,
 447,
 447,
 447,
 448,
 449,
 449,
 450,
 454,
 455,
 455,
 456,
 457,
 457,
 458,
 458,
 461,
 461,
 461,
 464,
 464,
 465,
 465,
 466,
 467,
 468,
 469,
 469,
 470,
 470,
 470,
 471,
 472,
 472,
 475,
 475,
 476,
 476,
 477,
 477,
 478,
 480,
 480,
 481,
 482,
 482,
 482,
 483,
 483,
 484,
 484,
 484,
 485,
 485,
 486,
 487,
 487,
 488,
 489,
 490,
 490,
 490,
 490,
 490,
 491,
 491,
 491,
 493,
 493,
 493,
 493,
 494,
 494,
 495,
 495,
 495,
 495,
 496,
 496,
 496,
 496,
 496,
 497,
 498,
 498,
 498,
 499,
 499,
 500,
 500,
 500,
 501,
 501,
 502,
 502,
 502,
 502,
 502,
 503

## 3000 genes

In [21]:
sort_arr(eig_val_from_lap(A,3000))

array([1.38007292e-12, 4.37185455e+02, 4.48120048e+02, ...,
       1.89835669e+03, 1.90240225e+03, 1.94330305e+03])

In [22]:
sort_arr(get_elem_in_diag(degree_mat_from_adj(A,3000)))

[438,
 449,
 515,
 519,
 520,
 522,
 525,
 531,
 534,
 546,
 553,
 557,
 560,
 562,
 574,
 575,
 576,
 576,
 577,
 577,
 578,
 580,
 583,
 587,
 590,
 591,
 591,
 592,
 594,
 595,
 596,
 597,
 598,
 600,
 600,
 601,
 603,
 606,
 610,
 610,
 612,
 614,
 614,
 614,
 617,
 617,
 618,
 618,
 620,
 620,
 625,
 625,
 630,
 631,
 631,
 632,
 632,
 634,
 634,
 635,
 635,
 635,
 636,
 637,
 639,
 639,
 640,
 641,
 641,
 642,
 643,
 644,
 644,
 644,
 644,
 648,
 649,
 650,
 653,
 654,
 654,
 656,
 657,
 663,
 664,
 666,
 666,
 668,
 669,
 669,
 670,
 671,
 673,
 673,
 674,
 675,
 675,
 676,
 676,
 676,
 682,
 682,
 682,
 683,
 684,
 685,
 685,
 686,
 687,
 688,
 688,
 690,
 690,
 692,
 693,
 693,
 693,
 693,
 693,
 694,
 696,
 696,
 696,
 697,
 697,
 699,
 699,
 699,
 700,
 701,
 702,
 702,
 703,
 703,
 703,
 705,
 705,
 706,
 706,
 706,
 707,
 709,
 710,
 710,
 710,
 712,
 713,
 714,
 714,
 714,
 715,
 715,
 715,
 715,
 716,
 717,
 717,
 719,
 719,
 721,
 722,
 722,
 723,
 724,
 724,
 724,
 724

## 4000 genes

In [23]:
sort_arr(eig_val_from_lap(A,4000))

array([-5.24287932e-12,  5.89153904e+02,  6.02049623e+02, ...,
        2.51336766e+03,  2.52233879e+03,  2.56332521e+03])

In [18]:
sort_arr(get_elem_in_diag(degree_mat_from_adj(A,4000)))

[590,
 603,
 670,
 677,
 684,
 684,
 687,
 690,
 693,
 701,
 702,
 708,
 724,
 725,
 726,
 728,
 730,
 731,
 747,
 756,
 759,
 761,
 763,
 763,
 763,
 766,
 766,
 771,
 772,
 774,
 775,
 777,
 779,
 782,
 783,
 786,
 788,
 788,
 788,
 789,
 790,
 790,
 794,
 794,
 794,
 795,
 797,
 800,
 802,
 806,
 809,
 811,
 811,
 815,
 820,
 820,
 821,
 821,
 821,
 822,
 823,
 825,
 828,
 829,
 829,
 830,
 830,
 832,
 834,
 835,
 836,
 836,
 838,
 839,
 839,
 839,
 839,
 840,
 842,
 842,
 845,
 846,
 847,
 849,
 850,
 851,
 851,
 851,
 855,
 855,
 856,
 857,
 859,
 859,
 861,
 864,
 865,
 866,
 866,
 868,
 869,
 871,
 871,
 872,
 873,
 875,
 875,
 876,
 880,
 881,
 883,
 884,
 884,
 885,
 886,
 889,
 889,
 889,
 890,
 893,
 893,
 895,
 895,
 895,
 896,
 897,
 898,
 898,
 900,
 903,
 903,
 903,
 907,
 907,
 907,
 907,
 910,
 910,
 912,
 912,
 913,
 914,
 915,
 915,
 916,
 916,
 916,
 917,
 917,
 918,
 919,
 920,
 921,
 921,
 921,
 922,
 922,
 922,
 923,
 923,
 923,
 925,
 926,
 927,
 928,
 929,
 930

## 5000 genes

In [25]:
sort_arr(eig_val_from_lap(A,5000))

array([-1.50956626e-12,  7.38171121e+02,  7.57049970e+02, ...,
        3.13637727e+03,  3.14933574e+03,  3.19732587e+03])

In [26]:
sort_arr(get_elem_in_diag(degree_mat_from_adj(A,5000)))

[739,
 758,
 853,
 854,
 857,
 861,
 864,
 864,
 869,
 877,
 881,
 884,
 894,
 901,
 905,
 913,
 914,
 918,
 919,
 934,
 935,
 945,
 945,
 951,
 958,
 958,
 959,
 961,
 961,
 964,
 965,
 966,
 968,
 971,
 971,
 973,
 974,
 975,
 975,
 985,
 985,
 986,
 987,
 987,
 987,
 987,
 990,
 990,
 991,
 993,
 996,
 996,
 999,
 999,
 1003,
 1006,
 1006,
 1011,
 1012,
 1014,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1023,
 1026,
 1029,
 1029,
 1031,
 1034,
 1034,
 1035,
 1043,
 1043,
 1045,
 1045,
 1046,
 1050,
 1051,
 1051,
 1052,
 1052,
 1053,
 1054,
 1055,
 1055,
 1055,
 1056,
 1056,
 1056,
 1058,
 1060,
 1061,
 1065,
 1066,
 1067,
 1069,
 1070,
 1071,
 1072,
 1074,
 1074,
 1076,
 1077,
 1078,
 1078,
 1078,
 1079,
 1079,
 1080,
 1081,
 1082,
 1085,
 1085,
 1085,
 1085,
 1089,
 1089,
 1092,
 1095,
 1096,
 1100,
 1100,
 1101,
 1101,
 1102,
 1103,
 1104,
 1104,
 1105,
 1105,
 1106,
 1106,
 1107,
 1109,
 1109,
 1110,
 1111,
 1112,
 1112,
 1112,
 1113,
 1113,
 1115,
 1119,
 1119,
 1119,
 1120,
 112

## 6000 genes

In [30]:
sort_arr(eig_val_from_lap(A,6000))

array([-1.20571598e-11,  8.66118585e+02,  8.81221161e+02, ...,
        3.75237561e+03,  3.81900000e+03,  3.81965768e+03])

In [31]:
sort_arr(get_elem_in_diag(degree_mat_from_adj(A,6000)))

[867,
 882,
 914,
 919,
 961,
 977,
 1011,
 1011,
 1015,
 1029,
 1032,
 1033,
 1040,
 1043,
 1044,
 1049,
 1057,
 1066,
 1076,
 1080,
 1082,
 1087,
 1093,
 1096,
 1099,
 1103,
 1110,
 1110,
 1118,
 1124,
 1125,
 1125,
 1129,
 1139,
 1140,
 1143,
 1152,
 1159,
 1162,
 1162,
 1163,
 1163,
 1163,
 1164,
 1169,
 1169,
 1173,
 1176,
 1176,
 1178,
 1178,
 1181,
 1186,
 1190,
 1191,
 1193,
 1195,
 1199,
 1199,
 1200,
 1201,
 1204,
 1209,
 1210,
 1210,
 1210,
 1214,
 1215,
 1215,
 1216,
 1219,
 1219,
 1219,
 1223,
 1225,
 1228,
 1229,
 1229,
 1230,
 1231,
 1233,
 1234,
 1235,
 1235,
 1237,
 1238,
 1239,
 1240,
 1245,
 1247,
 1247,
 1247,
 1248,
 1253,
 1255,
 1255,
 1256,
 1257,
 1259,
 1262,
 1263,
 1263,
 1263,
 1264,
 1266,
 1269,
 1269,
 1272,
 1273,
 1275,
 1276,
 1277,
 1278,
 1279,
 1280,
 1281,
 1284,
 1285,
 1285,
 1285,
 1286,
 1287,
 1287,
 1287,
 1287,
 1287,
 1288,
 1291,
 1291,
 1294,
 1295,
 1298,
 1299,
 1300,
 1302,
 1303,
 1304,
 1304,
 1305,
 1305,
 1305,
 1306,
 1306,
 1306

## 7000 genes

In [32]:
sort_arr(eig_val_from_lap(A,7000))

array([4.19472443e-11, 1.02707697e+03, 1.04023456e+03, ...,
       4.44400000e+03, 4.44466173e+03, 4.46131766e+03])

In [33]:
sort_arr(get_elem_in_diag(degree_mat_from_adj(A,7000)))

[1028,
 1041,
 1066,
 1079,
 1141,
 1143,
 1178,
 1194,
 1195,
 1201,
 1209,
 1219,
 1219,
 1221,
 1224,
 1228,
 1228,
 1241,
 1251,
 1255,
 1269,
 1273,
 1276,
 1278,
 1278,
 1281,
 1300,
 1303,
 1306,
 1310,
 1311,
 1319,
 1324,
 1327,
 1330,
 1334,
 1334,
 1340,
 1341,
 1344,
 1346,
 1348,
 1351,
 1353,
 1356,
 1357,
 1359,
 1359,
 1361,
 1362,
 1364,
 1365,
 1366,
 1367,
 1370,
 1370,
 1373,
 1382,
 1386,
 1389,
 1391,
 1391,
 1392,
 1393,
 1394,
 1395,
 1399,
 1404,
 1405,
 1406,
 1408,
 1409,
 1409,
 1412,
 1413,
 1413,
 1415,
 1419,
 1419,
 1419,
 1419,
 1420,
 1421,
 1422,
 1425,
 1425,
 1427,
 1430,
 1434,
 1435,
 1435,
 1437,
 1438,
 1439,
 1440,
 1441,
 1441,
 1443,
 1443,
 1444,
 1444,
 1446,
 1447,
 1449,
 1451,
 1452,
 1454,
 1456,
 1456,
 1458,
 1459,
 1460,
 1461,
 1462,
 1463,
 1463,
 1465,
 1466,
 1466,
 1474,
 1475,
 1477,
 1477,
 1477,
 1483,
 1485,
 1485,
 1486,
 1486,
 1487,
 1487,
 1489,
 1492,
 1494,
 1495,
 1495,
 1496,
 1496,
 1498,
 1498,
 1498,
 1498,
 1499,

In [34]:
sort_arr(eig_val_from_lap(A,8000))

array([            nan+7.96233421e-269j, 3.89393655e-308            +nanj,
                   nan+6.96249312e-308j, ...,
       9.38586668e-258+3.77352131e-258j, 1.09458971e-257+3.79090819e-275j,
       1.40578599e-257+2.89197887e-286j])